<a href="https://colab.research.google.com/github/franfram/AAR-DL/blob/main/aar_dl_CNN_TF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

TF tutorials:

https://www.tensorflow.org/tutorials/load_data/images
https://www.tensorflow.org/tutorials/keras/overfit_and_underfit
https://www.tensorflow.org/tutorials/images/classification
https://www.tensorflow.org/guide/data_performance
https://www.tensorflow.org/guide/keras
https://www.tensorflow.org/tutorials/keras/classification
https://www.tensorflow.org/tutorials/

The following comes from
https://www.tensorflow.org/tutorials/load_data/images

We will build a Convolutional Neural Network (CNN) from scratch using TensorFlow and Keras (TFs high level API).



In [1]:
# @title Setup (Needs cleaning)
!pip install fastai
!pip install --upgrade Pillow
from fastai import *

import numpy as np
import os
import PIL
import PIL.Image
from IPython.display import display
import tensorflow as tf
import tensorflow_datasets as tfds
import pathlib

import matplotlib.pyplot as plt
from pathlib import Path
from PIL import Image
import math
import random


print("TensorFlow version:", tf.__version__)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 20.0 MB/s eta 0:00:00
  Attempting uninstall: Pillow
    Found existing installation: Pillow 9.4.0
    Uninstalling Pillow-9.4.0:
      Successfully uninstalled Pillow-9.4.0


TensorFlow version: 2.12.0


In [2]:
# @title Get the data
def fetch_data():
  """Fetches required data from github repo and sets up the Files in google colab """

  if not os.path.exists('AAR-DL'):
    !git clone https://github.com/franfram/AAR-DL

  !mv AAR-DL/behaviour-images behaviour-images
  !rm -rf AAR-DL
  !rm -rf sample_data


fetch_data()

Cloning into 'AAR-DL'...
remote: Enumerating objects: 2453, done.
remote: Counting objects: 100% (2453/2453), done.
remote: Compressing objects: 100% (2325/2325), done.
remote: Total 2453 (delta 202), reused 2335 (delta 89), pack-reused 0
Receiving objects: 100% (2453/2453), 19.89 MiB | 12.51 MiB/s, done.
Resolving deltas: 100% (202/202), done.


In [32]:
top_folder = Path('behaviour-images')


holis = [file for subfolder in top_folder.iterdir() for file in subfolder.glob('*.png')]


holis == png_paths

True

In [ ]:
# @title get image paths

def get_image_paths(
    image_dir: str = 'behaviour-images' # name of parent folder with all image data
) -> tuple[list[pathlib.PosixPath], list[str], pathlib.PosixPath]:
  """Crawls through the folder containing the data and returns all the image paths, both in PosixPath and Str"""

  data_dir = pathlib.Path(image_dir)
  posix_paths = [file for subfolder in data_dir.iterdir() for file in subfolder.glob('*.png')]
  str_paths = [str(path) for path in posix_paths]

  return posix_paths, str_paths, data_dir


posix_paths, str_paths, data_dir = get_image_paths(image_dir = 'behaviour-images')

posix_paths, str_paths, data_dir

In [ ]:
# @title Display some of the image data

def display_random_images_with_titles(file_paths: list, N: int):
    """Open and display N randomly selected images from the given paths, with their subfolder names as the titles."""

    # Randomly select N file paths
    sampled_paths = random.sample(file_paths, N)

    # Calculate grid dimensions
    columns = math.ceil(math.sqrt(N))
    rows = math.ceil(N / columns)

    # Create a new figure
    fig, axes = plt.subplots(rows, columns, figsize=(15, 15))  # Adjust figsize for your needs

    # If only one row or column, ensure axes is 2D array
    if rows == 1:
        axes = axes.reshape(1, -1)
    if columns == 1:
        axes = axes.reshape(-1, 1)

    for ax, file_path in zip(axes.ravel(), sampled_paths):
        # Open the image using PIL
        image = Image.open(file_path)

        # Extract the subfolder name from the path
        subfolder_name = Path(file_path).parts[-2]

        # Display the image with the subfolder name as the title
        ax.imshow(image)
        ax.set_title(subfolder_name)
        ax.axis('off')

    # Hide any remaining axes
    for ax in axes.ravel()[N:]:
        ax.axis('off')

    plt.tight_layout()
    plt.show()


display_random_images_with_titles(str_paths, N = 8)


In [ ]:
# @title Load data using a Keras utility

# Create dataset

## define some parameters for the loader

batch_size = 32


img_width, img_height = Image.open(str_paths[0]).size # all the images are the same size


## create the training dataset
train_ds = tf.keras.utils.image_dataset_from_directory(
  'behaviour-images',
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size
)




In [ ]:
## create the validation dataset
val_ds = tf.keras.utils.image_dataset_from_directory(
  'behaviour-images',
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size
)


We can find the `class_names` attribute on these datasets in the following way:

In [ ]:
class_names = train_ds.class_names
print(class_names)

We can also iterate manually over the dataset andd retrieve batches of images:


In [ ]:
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

The `image_batch` is a tensor of the shape `(32, 3, 200, 3)`. This is a batch of 32 images of shape `3x200x3` (the last dimension refers to the channels RGB). The `label_batch` is a tensor of the shape `(32,)`, these are corresponding labels to the 32 images.





If we want, we can convert these tensors to `numpy.ndarray`s with `.numpy()`

In [ ]:
type(image_batch)

In [ ]:
image_batch.shape, image_batch[0].shape, image_batch[0][0].shape, image_batch[0, :, :, 1].shape

In [ ]:
image_batch[0, :, :, ] # WHY DO i HAVE 3 CHANNELS IF IS A GREY IMAGE? Answer: the 3 channels are the same, this is probably something default that TF DOES

probing that all 3 channels are the same

In [ ]:
a, b, c = image_batch[0, :, :,0], image_batch[0, :, :, 1], image_batch[0, :, :, 2]

In [ ]:
a, b
np.array_equal(a, b), np.array_equal(a, c)

ToDO: check if you have to do something different when you have 1 channel. (probably is just a 1D CNN?)

# Standarize the data

The RGB channel values are in the [0, 255] range. This is not ideal for a neural net, in general you should seed to make the input values small. We will standarize the values to be in the [0, 1] range.
We will do it here just to show how it works, but then we will include the normalization layer inside the model definition to simplify deployment.



In [ ]:
normalization_layer = tf.keras.layers.Rescaling(1./255)

normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))

image_batch, labels_batch = next(iter(normalized_ds))


image_batch, labels_batch # note that now image_batch is in the [0, 1] range but labels_batch remain unchanged
#e.g.



# Configure the dataset for performance
Let's make sure to use buffered prefetching so you can yield data from disk without having I/O become blocking. These are two important methods you should use when loading data:

  - Dataset.cache keeps the images in memory after they're loaded off disk during the first epoch. This will ensure the dataset does not become a bottleneck while training your model. If your dataset is too large to fit into memory, you can also use this method to create a performant on-disk cache.
  
  - Dataset.prefetch overlaps data preprocessing and model execution while training.

Interested readers can learn more about both methods, as well as how to cache data to disk in the Prefetching section of the Better performance with the tf.data API guide. (https://www.tensorflow.org/guide/data_performance)



In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

train_ds, val_ds

# Train the model

In [ ]:
num_classes = len(set(np.array(labels_batch)))


# define the model
model = tf.keras.Sequential([
    tf.keras.layers.Rescaling(1./255),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(padding='same'),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(padding='same'),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(padding='same'),
    # tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(num_classes)
])


model

In [ ]:
# compile the model
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True),
    metrics=['accuracy']
)

In [ ]:
# fit the model
model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=3
)

TO FIX LATER

The following comes from: https://www.tensorflow.org/tutorials/keras/classification  and   https://www.tensorflow.org/tutorials/keras/keras_tuner



In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential


In [ ]:
# @title Get the data
if not os.path.exists('AAR-DL'):
  !git clone https://github.com/franfram/AAR-DL

!mv AAR-DL/behaviour-images behaviour-images
!rm -rf AAR-DL
!rm -rf sample_data


In [ ]:

data_dir = pathlib.Path('behaviour-images')

png_paths = [file for subfolder in data_dir.iterdir() for file in subfolder.glob('*.png')]
image_count = len(png_paths)
image_count, png_paths



The following comes from:
https://www.tensorflow.org/tutorials/images/classification

CONTINUE WITH https://www.tensorflow.org/tutorials/images/cnn